### note: implement auc metric 

Run `chexpert_dev/distributed_pneumothorax.py`

### imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.vision import *
import fastai; fastai.__version__

'1.0.55.dev0'

In [3]:
chexpert_path = Path("../../data/chexpert/")
mimic_path = Path("../../data/mimic-cxr")

In [4]:
chexpert_path.ls()

[PosixPath('../../data/chexpert/models'),
 PosixPath('../../data/chexpert/CheXpert-v1.0.zip'),
 PosixPath('../../data/chexpert/CheXpert-v1.0-small')]

In [5]:
mimic_path.ls()

[PosixPath('../../data/mimic-cxr/train'),
 PosixPath('../../data/mimic-cxr/chx_mimic_valid_320.csv'),
 PosixPath('../../data/mimic-cxr/chx_mimic_train_320.csv'),
 PosixPath('../../data/mimic-cxr/valid'),
 PosixPath('../../data/mimic-cxr/valid_320'),
 PosixPath('../../data/mimic-cxr/train.csv'),
 PosixPath('../../data/mimic-cxr/train_320'),
 PosixPath('../../data/mimic-cxr/valid.csv')]

In [6]:
torch.cuda.set_device(3)

### databunch 

In [7]:
train_df = pd.read_csv(mimic_path/"chx_mimic_train_320.csv")
valid_df = pd.read_csv(mimic_path/"chx_mimic_valid_320.csv")

In [8]:
cols = ['Path', 'Pneumothorax']

In [13]:
# only pneumothorax
train_df, valid_df = train_df[cols], valid_df[cols]
train_valid_df = pd.concat([train_df, valid_df])

In [14]:
train_valid_df.Pneumothorax.value_counts()

-1.0    447007
 0.0    114601
 1.0     33898
Name: Pneumothorax, dtype: int64

In [15]:
train_valid_df.Pneumothorax.value_counts(normalize=True)

-1.0    0.750634
 0.0    0.192443
 1.0    0.056923
Name: Pneumothorax, dtype: float64

In [16]:
# binary classification
# train_valid_df = train_valid_df[train_valid_df.Pneumothorax != -1].reset_index(drop=True)
# train_valid_df.Pneumothorax.value_counts(normalize=True)

In [19]:
sz,bs=320,64
data_path = Path("../../data")

In [20]:
tfms = get_transforms()
pretrained = True

data = (ImageList.from_df(train_valid_df, data_path, cols=['Path'])
        .split_by_rand_pct(valid_pct=0.02, seed=42)
        .label_from_df(label_cls=CategoryList)
        .transform(tfms, size=sz)
        .databunch(bs=bs, num_workers=32))
data = data.normalize(imagenet_stats) if pretrained else data.normalize()

In [21]:
data.stats

([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [62]:
data.classes, data.c

([-1.0, 0.0, 1.0], 3)

In [63]:
# set data classes
data.classes = label_columns
data.classes, data.stats

(Index(['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
        'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
        'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
        'Fracture', 'Support Devices'],
       dtype='object'), ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))

In [22]:
# data.show_batch(3)

In [19]:
# xb,yb = data.one_batch()
# data.train_ds.y.c2i

### model

In [33]:
# from chexpert_dev.uignore import *

In [64]:
pretrained = True
# eps = 1e-4
# opt_func= partial(optim.Adam, betas=(0.9,0.99), eps=eps)

In [65]:
if pretrained: learn = cnn_learner(data, models.densenet121, pretrained=True).to_fp16()
else: learn = Learner(data, models.xresnet34(c_out=data.c)).to_fp16()
learn.metrics = [accuracy]

In [66]:
# learn.lr_find()

In [67]:
# lr = learn.recorder.plot(suggestion=True)

In [ ]:
lr = 3e-2
if not pretrained:
    # training from scratch
    learn.fit_one_cycle(10, lr)
else:
    # # transfer learning
    learn.fit_one_cycle(5, lr)
    learn.freeze_to(-1)
    learn.fit_one_cycle(5, slice(lr/5))
    learn.unfreeze()
    learn.fit_one_cycle(5, slice(lr/10))

In [ ]:
if pretrained: learn.to_fp32().save(f"chx-mimic-only-pneumo-densenet121-{sz}")
else: learn.to_fp32().save(f"chx-mimic-only-pneumo-xresnet34-{sz}"